In [4]:
# !py -m pip install pycalphad
# !py -m pip install numpy
# !py -m pip install tqdm
# !py -m pip install ipywidgets

from pycalphad import Database, equilibrium, variables as v
import random
import numpy as np
import time
from tqdm import tqdm
from pycalphad.core.utils import filter_phases, unpack_components
import warnings
warnings.filterwarnings("ignore")

# constants:
P = 101325
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)
# TODO: check 'PM' param
all_components = ['LI', 'BE', 'NA', 'MG', 'AL', 'K', 'CA', 'SC', 'TI', 'V', 'CR', 'MN', 'FE',
                  'CO', 'NI', 'CU', 'ZN', 'GA', 'Y', 'ZR', 'NB', 'MO', 'TC',
                  'RH', 'PD', 'AG', 'CD', 'IN', 'SN', 'BA', 'LA', 'CE', 'PR', 'ND', 'SM', 'EU',
                  'GD', 'TB', 'DY', 'HO', 'ER', 'HF', 'TA', 'W', 'RE', 'OS', 'IR',
                  'PT', 'AU', 'HG', 'TL', 'PB', 'BI', 'C', 'SI']
all_phases = list(db.phases)
all_phases.sort()
all_components.sort()

In [ ]:
# phase result parser
def flattenArray(arr):
    result = []
    for item in arr.flat:
        if isinstance(item, np.ndarray):
            result.extend(flattenArray(item))
        else:
            result.append(item)
    return result

# Get all possible phases for that components
def getPossiblePhases(db, components):
    comps = list(components)
    comps = unpack_components(db, comps)
    return filter_phases(db, comps)

# TODO: fix ordered_phases
def filterOrderedPhases(dbf, candidate_phases=None):
    if candidate_phases == None:
        candidate_phases = dbf.phases.keys()
    else:
        candidate_phases = set(
            candidate_phases).intersection(dbf.phases.keys())
    # all_phases = [{"name": phase, "ordered": dbf.phases[phase].model_hints.get('ordered_phase'),'dis':dbf.phases[phase].model_hints.get('disordered_phase')} for phase in candidate_phases if dbf.phases[phase].model_hints.get('ordered_phase') != None]
    # disordered_phases = [dbf.phases[phase].model_hints.get('disordered_phase') for phase in candidate_phases]
    ordered_phases = [dbf.phases[phase].model_hints.get(
        'ordered_phase') for phase in candidate_phases]

    phases = [phase for phase in candidate_phases if
              (phase not in ordered_phases)]
    #  or (phase in disordered_phases and
    # dbf.phases[phase].model_hints.get('ordered_phase') not in candidate_phases)
    return sorted(phases)


def getPhases(int_mask, all_phases=all_phases):
    return [all_phases[i] for i in range(len(all_phases)) if (int_mask & (1 << i)) > 0]


def phasesToMask(phases, all_phases=all_phases):
    phases.sort()
    mask = [1 if phase in phases else 0 for phase in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)


def phaseToMask(phase, all_phases=all_phases):
    mask = [1 if phase == phase_ else 0 for phase_ in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)

def compToMask(components, all_components=all_components):
    components.sort()
    mask = [1 if comp in components else 0 for comp in all_components][::-1]
    return int(''.join(map(str, mask)), 2)

def getComponents(int_mask, all_components=all_components):
    return [all_components[i] for i in range(len(all_components)) if (int_mask & (1 << i)) > 0]

def equilibriumRow(conditions, components, temp, amounts, iter):
    calc_components = components.copy()
    calc_components.append('VA')
    row_values = {}
    founded_phases = []
    possible_phases = getPossiblePhases(db, components)
    # possible_phases = filterOrderedPhases(db, possible_phases)

    tic = time.perf_counter()
    try:
        eq = equilibrium(db, calc_components, possible_phases,
                         conditions, verbose=False)
        founded_phases = flattenArray(np.array(eq.Phase))
    except ValueError as error:
        row_values['Error'] = error
    toc = time.perf_counter()

    row_values = {'iter': iter, 'T': temp, 'amounts': amounts, 'Components': compToMask(components), 'phases': phasesToMask(founded_phases),
                  'ellapsed_time': toc - tic, 'possible_phases': phasesToMask(possible_phases), 'P': P, 'Error': ''} | row_values

    return row_values

def parseConditions(temp, amounts, components, P=P):
    conditions = {v.T: temp, v.P: P}

    for i in range(len(components)):
        if i != 0:
            conditions[v.X(components[i])] = amounts[i]

    return conditions

def getAmounts(components):
    amounts = [0.05]*len(components)
    for i in range(len(amounts)):
        amounts[i] += random.uniform(0, min(1 - sum(amounts), 0.3))
    amounts[-1] += 1 - sum(amounts)
    return amounts

# TODO: make random temperature more stable diffuse
def getTemp():
    return random.randint(300, 3000)


In [ ]:
import csv

warnings.filterwarnings("ignore")

random.seed(42)
line = {}

# 
with open('DataSets/equilibrium_result.csv', "a") as csv_file:
    header = ['iter', 'T','amounts','Components','phases','ellapsed_time','possible_phases','P', 'Error']
    writer = csv.writer(csv_file, delimiter=';', lineterminator = '\n')
    # writer.writerow(header)
    rnd_state = random.getstate()
    for j in range(1):
        # TODO: iterate over components
        components = ['ZR', 'MO', 'W', 'TA', 'V', 'CR', 'CO', 'NI']
        print(f'{j} - {components}')
        for i in tqdm(range(1000)):
            temp = getTemp()
            amounts = getAmounts(components)
            if i < 500:
                continue
            conditions = parseConditions(temp, amounts, components)
            line = equilibriumRow(conditions, components, temp, amounts, i) 
            writer.writerow(list(line.values()))
    # TODO: Save random state in a file and continue from state
    print('Done!')

0 - ['PB', 'CU', 'TA', 'SC', 'SN', 'OS', 'LI', 'ER']
100%|██████████| 30/30 [00:00<00:00, 30002.17it/s]
1 - ['DY', 'SM', 'RH', 'FE', 'ND', 'NB', 'W', 'BA']
100%|██████████| 30/30 [00:00<00:00, 29973.59it/s]
2 - ['SN', 'TL', 'HF', 'SM', 'PT', 'CA', 'PD', 'AU']
100%|██████████| 30/30 [00:00<?, ?it/s]
3 - ['SI', 'V', 'Y', 'SC', 'CE', 'NB', 'AU', 'CD']
100%|██████████| 30/30 [00:00<00:00, 30016.49it/s]
4 - ['CE', 'SM', 'NA', 'BE', 'BI', 'V', 'TL', 'MG']
100%|██████████| 30/30 [00:00<?, ?it/s]
5 - ['BA', 'SC', 'NA', 'IR', 'PB', 'C', 'TA', 'NI']
100%|██████████| 30/30 [00:00<?, ?it/s]
6 - ['HF', 'NA', 'EU', 'W', 'SM', 'LA', 'ZR', 'MO']
100%|██████████| 30/30 [00:00<00:00, 30124.28it/s]
7 - ['GD', 'IR', 'RE', 'ER', 'W', 'CD', 'NA', 'CE']
100%|██████████| 30/30 [00:00<00:00, 30016.49it/s]
8 - ['BA', 'RH', 'AG', 'SN', 'CO', 'BI', 'NB', 'MG']
100%|██████████| 30/30 [00:00<00:00, 30009.33it/s]
9 - ['PB', 'CO', 'NB', 'GD', 'CD', 'CR', 'HG', 'MN']
100%|██████████| 30/30 [00:00<00:00, 29987.87it/s]
10 - ['BE', 'FE', 'TA', 'PD', 'RH', 'ZR', 'NB', 'MO']
100%|██████████| 30/30 [00:00<00:00, 30030.82it/s]
11 - ['HF', 'TB', 'SN', 'GD', 'CD', 'CR', 'ZN', 'AG']
100%|██████████| 30/30 [00:00<?, ?it/s]
12 - ['PR', 'C', 'AG', 'ND', 'SI', 'IR', 'PB', 'SC']
100%|██████████| 30/30 [00:00<00:00, 30052.33it/s]
13 - ['EU', 'HO', 'GA', 'BA', 'ND', 'ER', 'PB', 'SC']
100%|██████████| 30/30 [00:00<?, ?it/s]
14 - ['SM', 'CA', 'OS', 'AG', 'SC', 'TB', 'RE', 'BE']
100%|██████████| 30/30 [00:00<?, ?it/s]
15 - ['AG', 'K', 'SC', 'DY', 'SN', 'HG', 'LI', 'IR']
100%|██████████| 30/30 [00:00<?, ?it/s]
16 - ['CR', 'AL', 'CD', 'TB', 'EU', 'SI', 'FE', 'TC']
100%|██████████| 30/30 [00:00<00:00, 30009.33it/s]
17 - ['RH', 'PD', 'IN', 'SM', 'MO', 'SN', 'SI', 'MG']
100%|██████████| 30/30 [00:00<00:00, 29909.47it/s]
18 - ['IR', 'CR', 'K', 'NB', 'NA', 'CD', 'CO', 'CU']
100%|██████████| 30/30 [00:00<?, ?it/s]
19 - ['AU', 'MO', 'K', 'TA', 'NI', 'CA', 'AG', 'HF']
100%|██████████| 30/30 [00:00<?, ?it/s]
20 - ['W', 'NI', 'AU', 'HO', 'BI', 'CE', 'PT', 'TL']
100%|██████████| 30/30 [00:00<00:00, 30081.07it/s]
21 - ['CA', 'V', 'CE', 'SM', 'NA', 'NB', 'CR', 'CO']
100%|██████████| 30/30 [00:00<?, ?it/s]
22 - ['ZR', 'MN', 'SM', 'SI', 'BI', 'ND', 'LA', 'CU']
100%|██████████| 30/30 [00:00<00:00, 29966.45it/s]
23 - ['BA', 'ZN', 'BE', 'NB', 'SM', 'AL', 'CR', 'K']
100%|██████████| 30/30 [00:00<00:00, 29995.02it/s]
24 - ['Y', 'C', 'ZR', 'CO', 'SN', 'FE', 'ND', 'BE']
100%|██████████| 30/30 [12:56<00:00, 25.89s/it]
25 - ['TL', 'ND', 'AL', 'CA', 'TC', 'SI', 'MO', 'CR']
100%|██████████| 30/30 [08:53<00:00, 17.79s/it]
26 - ['ND', 'IN', 'CA', 'BA', 'TI', 'SC', 'Y', 'MN']
100%|██████████| 30/30 [06:07<00:00, 12.24s/it]
27 - ['MN', 'AG', 'ER', 'TC', 'AL', 'TI', 'GA', 'SM']
100%|██████████| 30/30 [08:58<00:00, 17.96s/it]
28 - ['ER', 'CU', 'MN', 'EU', 'LA', 'DY', 'HF', 'MG']
100%|██████████| 30/30 [08:08<00:00, 16.28s/it]
29 - ['DY', 'C', 'IN', 'HG', 'CR', 'AG', 'LA', 'SC']
100%|██████████| 30/30 [07:23<00:00, 14.79s/it]
30 - ['MO', 'SN', 'LA', 'TB', 'CD', 'AL', 'PT', 'CA']
100%|██████████| 30/30 [10:13<00:00, 20.44s/it]
31 - ['V', 'GD', 'SI', 'OS', 'MN', 'CD', 'PD', 'NB']
100%|██████████| 30/30 [12:28<00:00, 24.94s/it]
32 - ['AU', 'GA', 'CU', 'CE', 'MG', 'AL', 'RE', 'Y']
100%|██████████| 30/30 [16:32<00:00, 33.10s/it]
33 - ['NA', 'Y', 'BE', 'CA', 'PD', 'MG', 'TB', 'GA']
100%|██████████| 30/30 [09:37<00:00, 19.25s/it]
34 - ['DY', 'BA', 'ZR', 'AL', 'TI', 'SM', 'MG', 'GA']
100%|██████████| 30/30 [17:30<00:00, 35.03s/it]
35 - ['RE', 'C', 'PT', 'K', 'MG', 'CU', 'PD', 'IN']
100%|██████████| 30/30 [14:56<00:00, 29.88s/it]
36 - ['ZR', 'ZN', 'PT', 'NA', 'CA', 'GA', 'SM', 'NI']
100%|██████████| 30/30 [18:38<00:00, 37.29s/it]
37 - ['MN', 'TI', 'IR', 'PT', 'RH', 'AL', 'CR', 'HG']
100%|██████████| 30/30 [15:04<00:00, 30.14s/it]
38 - ['CR', 'CE', 'RE', 'SC', 'BA', 'ER', 'CA', 'W']
100%|██████████| 30/30 [03:24<00:00,  6.80s/it]
39 - ['PR', 'EU', 'RH', 'IN', 'NA', 'NI', 'SI', 'CR']
100%|██████████| 30/30 [09:26<00:00, 18.90s/it]
40 - ['CO', 'NA', 'AG', 'HG', 'BE', 'BA', 'CD', 'ND']
100%|██████████| 30/30 [04:42<00:00,  9.41s/it]
41 - ['PT', 'LA', 'OS', 'BE', 'IN', 'NB', 'SI', 'TC']
100%|██████████| 30/30 [09:36<00:00, 19.22s/it]
42 - ['TA', 'CU', 'HG', 'CD', 'GD', 'BE', 'AL', 'DY']
100%|██████████| 30/30 [08:30<00:00, 17.00s/it]
43 - ['CU', 'W', 'DY', 'RE', 'ND', 'MN', 'GA', 'C']
100%|██████████| 30/30 [11:11<00:00, 22.38s/it]
44 - ['TA', 'MO', 'SI', 'PR', 'NI', 'V', 'PD', 'GA']
100%|██████████| 30/30 [18:16<00:00, 36.54s/it]
45 - ['CU', 'PR', 'HF', 'ND', 'SN', 'SC', 'PB', 'GD']
100%|██████████| 30/30 [09:39<00:00, 19.30s/it]
46 - ['TC', 'ND', 'NI', 'CR', 'ZN', 'IR', 'SM', 'SN']
100%|██████████| 30/30 [15:37<00:00, 31.26s/it]
47 - ['CE', 'MN', 'C', 'TI', 'AU', 'CD', 'K', 'NA']
100%|██████████| 30/30 [13:53<00:00, 27.77s/it]
48 - ['ER', 'TA', 'PR', 'RE', 'EU', 'V', 'NB', 'PD']
100%|██████████| 30/30 [09:27<00:00, 18.92s/it]
49 - ['NB', 'V', 'ER', 'TA', 'TC', 'BA', 'GD', 'IR']
100%|██████████| 30/30 [05:23<00:00, 10.80s/it]
50 - ['ER', 'TC', 'CD', 'NA', 'RE', 'ND', 'LI', 'SC']
100%|██████████| 30/30 [02:56<00:00,  5.89s/it]
51 - ['CD', 'CO', 'BA', 'NA', 'CE', 'ZR', 'EU', 'PT']
100%|██████████| 30/30 [08:02<00:00, 16.08s/it]
52 - ['ND', 'HF', 'GD', 'SC', 'TA', 'LA', 'LI', 'EU']
100%|██████████| 30/30 [04:03<00:00,  8.12s/it]
53 - ['CD', 'NI', 'PT', 'Y', 'C', 'IR', 'CA', 'V']
100%|██████████| 30/30 [14:55<00:00, 29.86s/it]
54 - ['RH', 'PD', 'AL', 'SI', 'GA', 'CU', 'CR', 'CO']
100%|██████████| 30/30 [22:43<00:00, 45.45s/it]
55 - ['C', 'IR', 'EU', 'PB', 'SN', 'ND', 'NI', 'CR']
100%|██████████| 30/30 [16:13<00:00, 32.44s/it]
56 - ['GD', 'Y', 'AG', 'IN', 'LI', 'SI', 'CA', 'CE']
100%|██████████| 30/30 [12:18<00:00, 24.62s/it]
57 - ['SC', 'CD', 'HF', 'PT', 'HO', 'TC', 'RE', 'ND']
100%|██████████| 30/30 [04:57<00:00,  9.92s/it]
58 - ['ZR', 'ND', 'TC', 'TA', 'CO', 'IR', 'GD', 'DY']
100%|██████████| 30/30 [11:41<00:00, 23.38s/it]
59 - ['AG', 'W', 'DY', 'TA', 'SI', 'PD', 'PT', 'IN']
100%|██████████| 30/30 [20:12<00:00, 40.41s/it]
60 - ['OS', 'LA', 'FE', 'HF', 'ZR', 'CD', 'CO', 'CA']
100%|██████████| 30/30 [09:00<00:00, 18.02s/it]
61 - ['ZR', 'V', 'IR', 'FE', 'LA', 'K', 'CD', 'RH']
100%|██████████| 30/30 [08:30<00:00, 17.01s/it]
62 - ['HG', 'SC', 'EU', 'ZN', 'BI', 'PR', 'ND', 'BA']
100%|██████████| 30/30 [08:39<00:00, 17.33s/it]
63 - ['GD', 'DY', 'V', 'Y', 'TB', 'IR', 'MN', 'FE']
100%|██████████| 30/30 [10:16<00:00, 20.56s/it]
64 - ['AL', 'SN', 'PT', 'CD', 'AG', 'MN', 'CA', 'BI']
100%|██████████| 30/30 [19:28<00:00, 38.93s/it]
65 - ['TB', 'PT', 'TL', 'ND', 'K', 'BE', 'MG', 'DY']
100%|██████████| 30/30 [07:31<00:00, 15.04s/it]
66 - ['ZR', 'PB', 'HF', 'TL', 'CA', 'SC', 'ZN', 'HG']
100%|██████████| 30/30 [10:44<00:00, 21.49s/it]
67 - ['PB', 'MG', 'W', 'BI', 'PD', 'RE', 'DY', 'BE']
100%|██████████| 30/30 [10:47<00:00, 21.57s/it]
68 - ['NB', 'NA', 'HG', 'V', 'CO', 'W', 'IN', 'PD']
100%|██████████| 30/30 [14:02<00:00, 28.09s/it]
69 - ['GA', 'SI', 'MG', 'RH', 'W', 'AG', 'TI', 'MN']
100%|██████████| 30/30 [19:37<00:00, 39.26s/it]
70 - ['CR', 'W', 'PB', 'BE', 'IN', 'ZR', 'NA', 'OS']
100%|██████████| 30/30 [07:57<00:00, 15.92s/it]
71 - ['CD', 'NB', 'DY', 'AG', 'FE', 'CE', 'ZN', 'LI']
100%|██████████| 30/30 [14:40<00:00, 29.34s/it]
72 - ['SN', 'GD', 'TB', 'PT', 'OS', 'LI', 'FE', 'NA']
100%|██████████| 30/30 [11:18<00:00, 22.62s/it]
73 - ['HG', 'TI', 'C', 'TC', 'SM', 'CD', 'MO', 'SI']
100%|██████████| 30/30 [14:48<00:00, 29.63s/it]
74 - ['ER', 'AL', 'AG', 'HF', 'C', 'TC', 'MG', 'PT']
100%|██████████| 30/30 [22:35<00:00, 45.17s/it]
75 - ['PT', 'ER', 'CD', 'MO', 'SN', 'CO', 'TI', 'NB']
100%|██████████| 30/30 [22:30<00:00, 45.00s/it]
76 - ['GA', 'BI', 'GD', 'PD', 'W', 'CR', 'PT', 'AG']
100%|██████████| 30/30 [20:28<00:00, 40.95s/it]
77 - ['ZN', 'CA', 'SN', 'MO', 'BA', 'FE', 'SC', 'PB']
100%|██████████| 30/30 [18:06<00:00, 36.23s/it]
78 - ['MO', 'PD', 'PR', 'LI', 'CR', 'CU', 'IN', 'K']
100%|██████████| 30/30 [16:21<00:00, 32.70s/it]
79 - ['ND', 'BA', 'TI', 'TL', 'MG', 'NB', 'AU', 'ZR']
100%|██████████| 30/30 [20:26<00:00, 40.88s/it]
80 - ['LA', 'GD', 'FE', 'AU', 'CR', 'PB', 'CD', 'HG']
 77%|███████▋  | 23/30 [11:00<03:07, 26.75s/it]